# Data & Things (RUC F2023)

## Lecture 17: IoT and Data

# Monitoring IoT Sensor Data
## Web-based IoT Data Subscriber

This part of code simulates a Web-based IoT data subscriber that receives data sent out by a subscriber (see DT-F23-Lecture17_IoT_Publisher.ipynb) via a MQTT broker. Note a subscriber doesn't work without a subscriber or a broker.

See DT-F23-Lecture17_IoT_Publisher.ipynb for more details about the subscriber and broker.

We use the **dash** and **dash-bootstrap-components** modules to create the web application. You need to install them:

* pip install dash
* pip install dash-bootstrap-components

## 1. Initialization

In the first part, we are importing the modules as usual. By now, you're supposed to already know mqtt and how to use it.

The global variable *current_temperature* will store the temperature data.

In [22]:
import dash
from dash import html, dcc, Input, Output

import dash_bootstrap_components as dbc
import paho.mqtt.client as mqtt

global current_temperature
current_temperature = "NaN"

## 2. Subscription

The following code is similar to the countpart in DT-F23-Lecture17_IoT_Subscriber.ipynb. However, there are two tiny differences here:

* In *on_message()* function, we declare the global variable *current_temperature* again and assign the received temperature to it.
* Instead of loop_forever() , we used *loop_start()* that will prevent stucking in a loop and continue running following lines.

In [23]:
# -----------------------------------------------------------------------------
# MQTT Subscribe
# -----------------------------------------------------------------------------
mqttc = mqtt.Client()
mqttc.connect("mqtt.eclipseprojects.io", 1883, 60)

def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+ str(rc))
    mqttc.subscribe("myroom/temperature")

def on_message(client, userdata, msg):
    global current_temperature
    current_temperature = msg.payload.decode()

mqttc.on_connect = on_connect
mqttc.on_message = on_message

mqttc.loop_start()

Connected with result code 0


## 2. Web App Creation

The following code creates the Web app and decides the page layout.

In [24]:
# -----------------------------------------------------------------------------
# Defining Dash app
# -----------------------------------------------------------------------------
app = dash.Dash(external_stylesheets=[dbc.themes.DARKLY])

# -----------------------------------------------------------------------------
# Temperature card
# -----------------------------------------------------------------------------
card = dbc.Card(
    html.H4(id="temperature")
)

Below is the layout object. In a container component, we added dcc.Interval component that will fire the dynamic callback function periodically. We set interval to 3000 milliseconds (ms), i.e., 3 seconds.

H1 is a header, and Hr is a horizontal break. Creating rows and columns is out of the scope of this post, but dbc.Row and dbc.Col are simply used for wrapping our temperature data card.

In [25]:
# -----------------------------------------------------------------------------
# Application layout
# -----------------------------------------------------------------------------
app.layout = dbc.Container(
    [
        dcc.Interval(id='update', n_intervals=0, interval=1000*3),
        html.H1("Monitoring IoT Sensor Data with Plotly Dash"),
        html.Hr(),
        dbc.Row(dbc.Col(card, lg=4))
    ]
)

Next, we define a callback function that takes the update id from dcc.Interval component that we recently defined as input, and outputs the temperature id we placed in a card. So, what will the function actually return? A string that contains the live temperature we recieved from "myroom/temperature" at the brokder side.

In [26]:
# -----------------------------------------------------------------------------
# Callback for updating temperature data
# -----------------------------------------------------------------------------
@app.callback(
    Output('temperature', 'children'),
    Input('update', 'n_intervals')
)

def update_temperature(timer):
    return ("Temperature: " + str(current_temperature) + " at time " + str(timer))

Lastly, in the main function, run the app!

In [27]:
# -----------------------------------------------------------------------------
# Main function
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


#### Reference

* https://medium.com/@tolgahan.cepel/monitoring-iot-sensor-data-with-plotly-dash-6c50761c0840